In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv('exercise.csv', low_memory=False)

In [33]:
exercise_labels = []

# Duyệt qua từng hàng trong DataFrame và tạo nhãn kiểu số
for index, row in df.iterrows():
    exercise_type = row['ExerciseType']
    exercise_difficulty = row['DifficultyLevel']
    
    # Ánh xạ type và difficulty thành các giá trị kiểu số từ 1 đến 9
    label = int((exercise_type - 1) * 3 + exercise_difficulty)
    exercise_labels.append(label)

# Suy ngược type và difficulty từ label
# exercise_type = (label - 1) // 3 + 1
# exercise_difficulty = (label - 1) % 3 + 1

In [34]:
df['label'] = exercise_labels

In [35]:
df

,BodyFatPercentage,BMI,DifficultyLevel,ExerciseType,Weight,Height,TDEE,label
0,34.175859,43.004913,1,1,99.525640,188.490363,2037.861846,1
1,23.867976,6.857392,2,2,57.671151,153.756741,1430.794645,5
2,45.521355,31.618442,3,1,86.505216,172.921975,1811.584473,3
3,15.759591,40.841293,1,3,95.412002,100.791129,1441.913398,7
4,21.549028,48.150677,1,3,32.980361,126.497713,973.302434,7
...,...,...,...,...,...,...,...,...
4995,6.235790,11.982209,2,2,32.017780,147.290562,1103.863029,5
4996,22.050854,43.910584,1,3,49.729389,114.558121,1067.606768,7
4997,16.342209,17.543850,2,2,46.079741,141.988434,1210.656433,5
4998,32.630619,41.057386,3,1,31.270325,184.401221,1319.892368,3


In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% huấn luyện, 20% kiểm tra)
X = df[['BMI', 'BodyFatPercentage', 'Weight', 'Height','TDEE']]
y = df['label']  # Label là cột bạn muốn dự đoán

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Chọn mô hình Gradient Boosting
model_gb = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Huấn luyện mô hình
model_gb.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred_gb = model_gb.predict(X_test)

# Đánh giá mô hình
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print("Gradient Boosting:")
print(f"Mean Squared Error: {mse_gb}")
print(f"R-squared: {r2_gb}")




Gradient Boosting:
Mean Squared Error: 0.723837132141304
R-squared: 0.8670339976991515


In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Chọn mô hình Random Forest
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Huấn luyện mô hình
model_rf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred_rf = model_rf.predict(X_test)

# Đánh giá mô hình
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest:")
print(f"Mean Squared Error: {mse_rf}")
print(f"R-squared: {r2_rf}")

Random Forest:
Mean Squared Error: 0.7186488
R-squared: 0.8679870736782704


In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Tạo mô hình Random Forest Classifier với các hyperparameters bạn muốn
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Huấn luyện mô hình trên tập huấn luyện
model.fit(X_train, y_train)

# Dự đoán kết quả trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá độ chính xác của mô hình trên tập kiểm tra
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.317


In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Chọn mô hình Random Forest
model_rf = RandomForestRegressor(random_state=42)

# Định nghĩa các giá trị thử nghiệm cho các siêu tham số
param_grid = {
    'n_estimators': [100, 200, 300],  # Thử các số lượng cây
    'max_depth': [None, 10, 20, 30],  # Thử các độ sâu
    'min_samples_split': [2, 5, 10],  # Thử số mẫu tối thiểu để chia nút
    'min_samples_leaf': [1, 2, 4],  # Thử số mẫu tối thiểu trong lá
}

# Sử dụng Grid Search để tìm bộ tham số tốt nhất
grid_search = GridSearchCV(model_rf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Lấy ra bộ tham số tốt nhất
best_params = grid_search.best_params_

# Tạo mô hình mới với bộ tham số tốt nhất
best_model_rf = RandomForestRegressor(random_state=42, **best_params)

# Huấn luyện mô hình mới trên dữ liệu huấn luyện
best_model_rf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred_rf = best_model_rf.predict(X_test)

# Đánh giá mô hình
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest:")
print(f"Mean Squared Error: {mse_rf}")
print(f"R-squared: {r2_rf}")


Random Forest:
Mean Squared Error: 0.6951042261293252
R-squared: 0.8723121182559082


In [40]:
best_params

{'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 100}

In [41]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X = df[['BMI', 'BodyFatPercentage', 'Weight', 'Height', 'TDEE']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [42]:
# Chọn mô hình XGBoost và thiết lập các siêu tham số
model_xgb = xgb.XGBRegressor(n_estimators=100, random_state=42)

# Huấn luyện mô hình
model_xgb.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [43]:
# Dự đoán trên tập kiểm tra
y_pred_xgb = model_xgb.predict(X_test)

# Đánh giá mô hình
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost:")
print(f"Mean Squared Error: {mse_xgb}")
print(f"R-squared: {r2_xgb}")


XGBoost:
Mean Squared Error: 0.9277036586391296
R-squared: 0.8295845276074678


In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Chọn mô hình Random Forest và sử dụng các tham số đã chỉ định
model_rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=1,
    random_state=42
)

# Huấn luyện mô hình
model_rf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred_rf = model_rf.predict(X_test)

# Đánh giá mô hình
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest:")
print(f"Mean Squared Error: {mse_rf}")
print(f"R-squared: {r2_rf}")


Random Forest:
Mean Squared Error: 0.6951042261293252
R-squared: 0.8723121182559082
